## Set up matching DF's

In [61]:
import pandas as pd
# Set pandas to display all columns
pd.set_option('display.max_columns', None)
import os
import re

In [62]:
directory = "/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/Toto"
file_extension = ".csv"
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(file_extension.split('.')[-1]) and (file.startswith('toto') or file.startswith('unibet'))]
if not files:
    raise FileNotFoundError(f"No files found in {directory} with extension {file_extension}")
latest_file_toto = max(files, key=os.path.getmtime)

directory = "/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/unibet"
file_extension = ".csv"
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(file_extension.split('.')[-1]) and (file.startswith('toto') or file.startswith('unibet'))]
if not files:
    raise FileNotFoundError(f"No files found in {directory} with extension {file_extension}")
latest_file_unibet = max(files, key=os.path.getmtime)

In [63]:
latest_file_toto

'/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/Toto/totoAllSport2025-02-17 10:21:04.707777.csv'

In [64]:
toto_raw = pd.read_csv(f"{latest_file_toto}", index_col=0).drop_duplicates()
kambi_raw = pd.read_csv(f"{latest_file_unibet}", index_col=0).drop_duplicates()
# Apply the condition and update outcome_label
toto_raw_football = toto_raw[toto_raw['sport'] == 'Voetbal']
kambi_raw_football = kambi_raw[kambi_raw['sport'] == 'FOOTBALL']
# Adjust odds and line
kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
kambi_raw_football['odds'] = kambi_raw_football['odds'] / 1000

# Adjust toto outcome
toto_raw_football['Outcome SubType'] = toto_raw_football['Outcome SubType'].replace({'H': '1', 'A': '2'})

# List of women's competitions
women_competitions_toto = [
    'Portugal Campeonato Nacional, Vrouwen', 'Mexico League MX Vrouwen', 'Australië W-League', 'Italië Coppa Italia Vrouwen',
    'Scotland Women\'s Premier League', 'Nederland Eredivisie Vrouwen', 'England FA Cup Women', 'Engeland FA Super League Vrouwen',
    'Spain Primera División Vrouwen'
]

# Create 'sex' column based on the competition
toto_raw_football['sex'] = toto_raw_football['competition'].apply(
    lambda x: 'W' if x in women_competitions_toto else 'M'
)

# List of women's football competitions
women_competitions_kambi = [
    'A-League (D)', 'Premier League Dames', 'Campeonato Nacional Feminino', 'Liga MX Femenil (D)', 
    'Frauen-Bundesliga', 'Super League (D)', 'Primera División (D)', 'Coppa Italia (D)', 'Liga MX Femenil'

]

# Create 'sex' column
kambi_raw_football['sex'] = kambi_raw_football['group_name'].apply(
    lambda x: 'W' if x in women_competitions_kambi else 'M'
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/3251815356.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  kambi_raw = pd.read_csv(f"{latest_file_unibet}", index_col=0).drop_duplicates()
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/3251815356.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/3251815356.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

Below is a translation of the abbreviations for the column outcome types into their likely meanings in the context of betting odds:

- `--`: Undefined or No Outcome (e.g., no odds available).
- `AG`: Anytime Goal Scorer (player to score at any time in the match).
- `CS`: Correct Score (predict the exact final score of the event).
- `DC`: Double Chance (cover two outcomes, e.g., Home Win or Draw).
- `DN`: Draw No Bet (stake refunded if the match ends in a draw).
- `FS`: First Scorer (player to score the first goal/point in the event).
- `H1`: First Half Result (outcome at the end of the first half).
- `H2`: Second Half Result (outcome in the second half only).
- `HF`: Half-Time/Full-Time (predict the result at both half-time and full-time).
- `HH`: Head-to-Head (comparison between two participants, e.g., players or teams).
- `HL`: Handicap Line (spread betting; adjust the line to level the playing field).
- `LS`: Last Scorer (player to score the last goal/point in the event).
- `MH`: Match Handicap (spread betting for the entire match).
- `MR`: Match Result (predict the overall winner or draw for the match).
- `OE`: Odd/Even (predict whether the total points/goals scored will be odd or even).
- `WH`: Winning Half (which half will have the higher score).
- `WM`: Winning Margin (predict the margin by which a team/player will win).

In [65]:
# # filter down to suitable betting opps
# kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

# kambi_filtered_football = kambi_filtered_football
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football

# Remove duplicates in toto_filtered_football based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name', 'Odds (Decimal)']
)
# Remove duplicates in kambi_filtered_football based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label', 'line', 'odds']
)

# Replace 'Sport Club' with 'SC' in the 'event_name' column
kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/1733036263.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)


In [66]:
len(toto_filtered_football)

320773

In [67]:
len(kambi_raw_football)

179731

## Winnaar: Draw no Bet

In [68]:
from rapidfuzz import process, fuzz
import pandas as pd

# Filter kambi_filtered_football based on criterion_label
filtered_kambi_winnaar = kambi_filtered_football[
    kambi_filtered_football['criterion_label'].str.contains('Draw No Bet')
]

# Filter kambi_filtered_football based on criterion_label
filtered_toto_winnaar = toto_filtered_football[
    toto_filtered_football['Market Name'].str.contains('Draw No Bet')
]

# Preprocess strings: Remove extra whitespace and lowercase
def preprocess(text):
    return ' '.join(text.lower().split())

In [69]:
import unicodedata
from fuzzywuzzy import process, fuzz

# Replace '-' with a space and remove accents in the 'names' column
filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Create 'Team1' and 'Team2'
filtered_kambi_winnaar[['Team1', 'Team2']] = filtered_kambi_winnaar['event_name'].str.split(' vs ', expand=True)
filtered_toto_winnaar[['Team1', 'Team2']] = filtered_toto_winnaar['Event Name'].str.split(' vs ', expand=True)

kambi_events = filtered_kambi_winnaar['event_name'].tolist()

# # Function to find the best match based on Team1 and Team2 fuzzy matching
# def find_best_match(event_name):
#     # Extract Team1 and Team2 from the event_name
#     parts = event_name.split(' vs ')
#     if len(parts) == 2:
#         team1 = parts[0].strip()
#         team2 = parts[1].strip()

#         # Perform fuzzy matching for Team1 and Team2
#         def match_teams(team, kambi_team):
#             return fuzz.token_set_ratio(team, kambi_team) >= 80

#         # Check each event_name in kambi_events
#         for kambi_event in kambi_events:
#             kambi_parts = kambi_event.split(' vs ')
#             if len(kambi_parts) == 2:
#                 kambi_team1 = kambi_parts[0].strip()
#                 kambi_team2 = kambi_parts[1].strip()

#                 # Check if both teams match
#                 if match_teams(team1, kambi_team1) and match_teams(team2, kambi_team2):
#                     return kambi_event  # Return the matched event_name

#     return None  # Return None if no match found

from fuzzywuzzy import fuzz

# Function to find the best match based on Team1 and Team2 fuzzy matching
def find_best_match(event_name):
    # Extract Team1 and Team2 from the event_name
    parts = event_name.split(' vs ')
    if len(parts) == 2:
        team1 = parts[0].strip()
        team2 = parts[1].strip()

        # Perform fuzzy matching for Team1 and Team2 with all kambi events
        def match_teams(team, kambi_team):
            return fuzz.token_set_ratio(team, kambi_team)

        # Perform fuzzy matching for both Team1 and Team2
        kambi_matches = filtered_kambi_winnaar[
            (filtered_kambi_winnaar['Team1'].apply(lambda x: match_teams(team1, x) >= 65)) &
            (filtered_kambi_winnaar['Team2'].apply(lambda x: match_teams(team2, x) >= 65))
        ]
        
        # If there is a match, return the matched event_name and fuzzy match score
        if not kambi_matches.empty:
            # Get the best match score
            best_score_team1 = kambi_matches['Team1'].apply(lambda x: match_teams(team1, x)).max()
            best_score_team2 = kambi_matches['Team2'].apply(lambda x: match_teams(team2, x)).max()

            # Calculate the average score (or you can choose to return another metric)
            average_score = (best_score_team1 + best_score_team2) / 2

            # Get the matched event name
            matched_event_name = kambi_matches['event_name'].iloc[0]

            return matched_event_name, average_score  # Return the matched event and score

    return None, None  # Return None if no match found

# Apply matching function to toto_filtered_football
filtered_toto_winnaar[['matched_event', 'fuzzy_score']] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: pd.Series(find_best_match(x))
)

# # Apply matching function to filtered_toto_winnaar
# filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

# Check the results
filtered_toto_winnaar[['Event Name', 'matched_event']].head()

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/954081136.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/954081136.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/954081136.py:20: SettingWith

,Event Name,matched_event
event_id,,
6894810,Helmond Sport vs Vitesse,Helmond Sport vs Vitesse
6894810,Helmond Sport vs Vitesse,Helmond Sport vs Vitesse
6617816,FC Utrecht vs NAC,None
6617816,FC Utrecht vs NAC,None
6617816,FC Utrecht vs NAC,None


In [70]:
import unicodedata
from fuzzywuzzy import process, fuzz

# Replace '-' with a space and remove accents in the 'names' column
filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Create 'Team1' and 'Team2'
filtered_kambi_winnaar[['Team1', 'Team2']] = filtered_kambi_winnaar['event_name'].str.split(' vs ', expand=True)
filtered_toto_winnaar[['Team1', 'Team2']] = filtered_toto_winnaar['Event Name'].str.split(' vs ', expand=True)

# Function to find the best match based on Team1 and Team2 fuzzy matching
def find_best_match(row):
    event_name = row['Event Name']
    start_time = row['start_time']
    
    parts = event_name.split(' vs ')
    if len(parts) == 2:
        team1 = parts[0].strip()
        team2 = parts[1].strip()

        # First subset based on matching start_time
        time_matched_events = filtered_kambi_winnaar[
            filtered_kambi_winnaar['start_time'] == start_time
        ]

        if not time_matched_events.empty:
            def match_teams(team, kambi_team):
                return fuzz.token_set_ratio(team, kambi_team)

            # Perform fuzzy matching for both Team1 and Team2 on time-matched subset
            kambi_matches = time_matched_events[
                (time_matched_events['Team1'].apply(lambda x: match_teams(team1, x) >= 65)) &
                (time_matched_events['Team2'].apply(lambda x: match_teams(team2, x) >= 65))
            ]
            
            if not kambi_matches.empty:
                best_score_team1 = kambi_matches['Team1'].apply(lambda x: match_teams(team1, x)).max()
                best_score_team2 = kambi_matches['Team2'].apply(lambda x: match_teams(team2, x)).max()
                average_score = (best_score_team1 + best_score_team2) / 2
                matched_event_name = kambi_matches['event_name'].iloc[0]
                return matched_event_name, average_score

    return None, None

# Apply matching function to filtered_toto_winnaar
filtered_toto_winnaar[['matched_event', 'fuzzy_score']] = filtered_toto_winnaar.apply(
    lambda row: pd.Series(find_best_match(row)),
    axis=1
)

# Check the results
filtered_toto_winnaar[['Event Name', 'matched_event']].head()

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/3094572401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/3094572401.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/3094572401.py:20: SettingW

,Event Name,matched_event
event_id,,
6894810,Helmond Sport vs Vitesse,Helmond Sport vs Vitesse
6894810,Helmond Sport vs Vitesse,Helmond Sport vs Vitesse
6617816,FC Utrecht vs NAC,None
6617816,FC Utrecht vs NAC,None
6617816,FC Utrecht vs NAC,None


In [71]:
filtered_toto_winnaar

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex,Team1,Team2,matched_event,fuzzy_score
event_id,,,,,,,,,,,,,,,,,,,
6894810,Helmond Sport vs Vitesse,Draw No Bet,Helmond Sport,1.56,14,25,DN,1,Voetbal,Nederland Eerste Divisie,Helmond Sport vs Vitesse,Helmond Sport,Vitesse,2025-02-21T19:00:00Z,M,Helmond Sport,Vitesse,Helmond Sport vs Vitesse,100.0
6894810,Helmond Sport vs Vitesse,Draw No Bet,Vitesse,2.28,32,25,DN,2,Voetbal,Nederland Eerste Divisie,Helmond Sport vs Vitesse,Helmond Sport,Vitesse,2025-02-21T19:00:00Z,M,Helmond Sport,Vitesse,Helmond Sport vs Vitesse,100.0
6617816,FC Utrecht vs NAC,Draw No Bet,NAC,4.00,3,1,DN,2,Voetbal,Nederland Eredivisie,FC Utrecht vs NAC,FC Utrecht,NAC,2025-03-01T15:30:00Z,M,FC Utrecht,NAC,None,NaN
6617816,FC Utrecht vs NAC,Draw No Bet,FC Utrecht,1.20,1,5,DN,1,Voetbal,Nederland Eredivisie,FC Utrecht vs NAC,FC Utrecht,NAC,2025-03-01T15:30:00Z,M,FC Utrecht,NAC,None,NaN
6617816,FC Utrecht vs NAC,1e Helft: Draw No Bet,FC Utrecht,1.26,13,50,DN,1,Voetbal,Nederland Eredivisie,FC Utrecht vs NAC,FC Utrecht,NAC,2025-03-01T15:30:00Z,M,FC Utrecht,NAC,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6559975,FC Wil vs FC Thun,Draw No Bet,FC Wil,2.35,27,20,DN,1,Voetbal,Zwitserland Challenge League,FC Wil vs FC Thun,FC Wil,FC Thun,2025-02-23T13:15:00Z,M,FC Wil,FC Thun,None,NaN
6650927,Norrkoping vs Osters,1e Helft: Draw No Bet,Norrköping,1.47,47,100,DN,1,Voetbal,Zweden Allsvenskan,Norrköping vs Östers,Norrköping,Östers,2025-03-30T14:30:00Z,M,Norrkoping,Osters,None,NaN
6650927,Norrkoping vs Osters,1e Helft: Draw No Bet,Östers,2.50,6,4,DN,2,Voetbal,Zweden Allsvenskan,Norrköping vs Östers,Norrköping,Östers,2025-03-30T14:30:00Z,M,Norrkoping,Osters,None,NaN


In [72]:
filtered_kambi_winnaar

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex,Team1,Team2
128,2494341982,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022544815,3645082694,1,1,1.38,NaN,Sporting San Jose FC,1,2025-02-14T22:22:02Z,4/11,-265.0,OPEN,DISABLED,NaN,NaN,Sporting San Jose FC vs Santos de Guapiles,FOOTBALL,Primera Division Costa Rica,2025-02-18T01:00:00Z,M,Sporting San Jose FC,Santos de Guapiles
129,2494341982,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022544815,3645082702,2,2,2.80,NaN,Santos de Guápiles,2,2025-02-14T22:22:02Z,9/5,180.0,OPEN,DISABLED,NaN,NaN,Sporting San Jose FC vs Santos de Guapiles,FOOTBALL,Primera Division Costa Rica,2025-02-18T01:00:00Z,M,Sporting San Jose FC,Santos de Guapiles
161,2494341991,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1022544815,3645082752,1,1,1.48,NaN,Sporting San Jose FC,1,2025-02-14T22:22:02Z,12/25,-210.0,OPEN,ENABLED,NaN,NaN,Sporting San Jose FC vs Santos de Guapiles,FOOTBALL,Primera Division Costa Rica,2025-02-18T01:00:00Z,M,Sporting San Jose FC,Santos de Guapiles
162,2494341991,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1022544815,3645082759,2,2,2.43,NaN,Santos de Guápiles,2,2025-02-14T22:22:02Z,7/5,143.0,OPEN,ENABLED,NaN,NaN,Sporting San Jose FC vs Santos de Guapiles,FOOTBALL,Primera Division Costa Rica,2025-02-18T01:00:00Z,M,Sporting San Jose FC,Santos de Guapiles
238,2494340266,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022627061,3645074860,1,1,1.38,NaN,Club Guaraní,1,2025-02-17T08:51:33Z,4/11,-265.0,OPEN,DISABLED,NaN,NaN,Club Guarani vs Deportivo Recoleta,FOOTBALL,Primera Paraguay,2025-02-21T21:15:00Z,M,Club Guarani,Deportivo Recoleta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193173,2494894952,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021350167,3647239694,2,2,1.83,NaN,Adelaide United,2,2025-02-16T22:25:41Z,41/50,-121.0,OPEN,DISABLED,NaN,NaN,Western United FC vs Adelaide United,FOOTBALL,A-League,2025-02-23T06:00:00Z,M,Western United FC,Adelaide United
193205,2494894978,1001421321,Draw No Bet - 2e Helft,Draw No Bet - 2nd Half,GOALS,NaN,2,Wedstrijd,Match,1021350167,3647239749,1,1,1.76,NaN,Western United FC,1,2025-02-16T22:25:41Z,3/4,-132.0,OPEN,DISABLED,NaN,NaN,Western United FC vs Adelaide United,FOOTBALL,A-League,2025-02-23T06:00:00Z,M,Western United FC,Adelaide United
193206,2494894978,1001421321,Draw No Bet - 2e Helft,Draw No Bet - 2nd Half,GOALS,NaN,2,Wedstrijd,Match,1021350167,3647239753,2,2,1.78,NaN,Adelaide United,2,2025-02-16T22:25:41Z,10/13,-129.0,OPEN,DISABLED,NaN,NaN,Western United FC vs Adelaide United,FOOTBALL,A-League,2025-02-23T06:00:00Z,M,Western United FC,Adelaide United
193286,2494895032,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021350167,3647239862,1,1,1.84,NaN,Western United FC,1,2025-02-16T22:25:40Z,5/6,-120.0,OPEN,DISABLED,NaN,NaN,Western United FC vs Adelaide United,FOOTBALL,A-League,2025-02-23T06:00:00Z,M,Western United FC,Adelaide United


In [73]:
# Get unique records from 'Event Name' and 'matched_event'
matched_events = filtered_toto_winnaar[['Event Name', 'matched_event', 'fuzzy_score']].drop_duplicates()

# # Display the result
# matched_events[~matched_events['matched_event'].isna()].sort_values(by='fuzzy_score', ascending=False).to_csv('test_match.csv')

In [74]:
def standardize_draw_no_bet(value):
    if '1e Helft' in value:
        return 'Draw No Bet - 1e Helft'
    elif '2e Helft' in value:
        return 'Draw No Bet - 2e Helft'
    elif 'Draw No Bet' in value:
        return 'Draw No Bet'
    else:
        return value  # Keep original if no match

# Apply the transformation to the relevant columns
filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)

# Merge the DataFrames using the standardized column
merged_winnaar = pd.merge(
    filtered_toto_winnaar,
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label', 'sex', 'start_time'],
    right_on=['event_name', 'standardized_label', 'sex', 'start_time'], 
    how='inner'  # Perform an inner join
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/3027481356.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/3027481356.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)


In [75]:
print(len(merged_winnaar.drop_duplicates()))

4396


In [76]:
# len(merged_winnaar['Event Name'].unique())

In [77]:
# # Filter rows in toto_filtered_football where Market Name == 'Winnaar'
# toto_filtered_football = toto_filtered_football[toto_filtered_football['Market Name'].str.contains('Draw No Bet')]

# Perform a left join on the matched_event_name (from toto) and event_name (from kambi)
merged_df_winnaar = filtered_toto_winnaar.merge(
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label', 'sex', 'start_time'], 
    right_on=['event_name', 'standardized_label', 'sex', 'start_time'], 
    how='left'
)

merged_df_winnaar = merged_df_winnaar[(merged_df_winnaar['matched_event'].notnull())&(merged_df_winnaar['Outcome SubType'] != merged_df_winnaar['outcome_label'])]

## Over/Under

In [78]:
# # filter down to suitable betting opps
# kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

# kambi_filtered_football = kambi_raw_football
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football

# Remove duplicates in toto_filtered_basketbal based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_basketbal based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

In [79]:
kambi_filtered_football_overunder = kambi_filtered_football[(kambi_filtered_football['outcome_english_label'].str.contains('Over')) |
    (kambi_filtered_football['outcome_english_label'].str.contains('Under'))
]
toto_filtered_football_overunder = toto_filtered_football[
    (toto_filtered_football['Outcome Name'].str.contains('Over')) |
    (toto_filtered_football['Outcome Name'].str.contains('Under')) |
    (toto_filtered_football['Outcome Name'].str.match(r'^\d+\+$')) |  # matches strings that ONLY contain digit(s) followed by '+'
    (toto_filtered_football['Outcome Name'].str.contains(r'\d{1,2} of meer')) &
    ~(toto_filtered_football['Outcome Name'].str.contains(' en ')) &  # excludes strings containing ' en '
    ~(toto_filtered_football['Outcome Name'].str.contains('&'))  # excludes strings containing '&'
]

In [80]:
toto_filtered_football_overunder[(toto_filtered_football_overunder['Event Name'].str.contains('Rakow Czestochowa vs GKS Katowice')) & (toto_filtered_football_overunder['Market Name'].str.contains('Aantal Goals   Over/Under 1.5'))]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex
event_id,,,,,,,,,,,,,,,


In [81]:
kambi_raw_football[(kambi_raw_football['event_id']==1020968352)& (kambi_raw_football['criterion_label']=='Totaal Aantal Doelpunten')]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex


In [82]:
import unicodedata

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_english_label'] = kambi_filtered_football_overunder['criterion_english_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Event Name'] = toto_filtered_football_overunder['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Market Name'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

kambi_events = kambi_filtered_football_overunder['event_name'].tolist()

# # Function to find the best match for 'Event Name' in toto_filtered_football
# def find_best_match(event_name):
#     # Use a token-based similarity for better handling of substrings
#     result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=100)
#     if result is None:  # No match found
#         return None
#     match, score = result
#     return match


# # Apply matching function to toto_filtered_football
# toto_filtered_football_overunder['matched_event'] = toto_filtered_football_overunder['Event Name'].apply(find_best_match)
# Perform a left join to bring in 'matched_event' and 'fuzzy_score' from matched_events
toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
    matched_events,
    on='Event Name',  # Join on the 'Event Name' column
    how='left'  # Ensure all rows in toto_filtered_football_overunder are retained
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/2219985270.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/2219985270.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_r

In [83]:
# toto_filtered_football_overunder.to_csv('Data/Testing/toto_filtered_football_overunder.csv')
# kambi_filtered_football_overunder.to_csv('Data/Testing/kambi_filtered_football_overunder.csv')  

In [84]:
# Create 'OverUnderType'
kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
        lambda x: 'Goals' if ('Doelpunten' in x and 'Resultaat' not in x and 'Doelpuntenmaker' not in x) 
        else 'Team schoten op doel' if ('Totaal Aantal Schoten op Doel door' in x and ' & ' not in x) 
        else 'Wedstrijd schoten op doel' if ('Totaal Aantal Schoten op Doel' in x and ' & ' not in x)
        else 'Team schoten' if ('Totaal Aantal Schoten door' in x and ' & ' not in x) 
        else 'Wedstrijd schoten' if ('Totaal Aantal Schoten' in x and ' & ' not in x)
        else 'Speler schoten op doel' if ('Schoten van Speler op Doel' in x and ' & ' not in x)
        else 'Speler schoten' if ('Schoten van Speler' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
         else 'Dubbele Kans' if ('Dubbele Kans' in x or ' en ' in x or ' & ' in x)
        else 'other'
    )

def determine_over_under_time(label):
    label_lower = label.lower()

    if '1e helft' in label_lower:
        return '1e Helft'
    elif '2e helft' in label_lower:
        return '2e Helft'

    # Check if the label contains a time range in the format 'XX:XX YY:YY'
    time_match = re.search(r'\b\d{1,2}:\d{2} \d{1,2}:\d{2}\b', label)
    if time_match:
        return time_match.group()  # Return the matched time range
    
    return 'Full Time'

# Apply the function to the column
kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)

# Create 'Team1' and 'Team2'
kambi_filtered_football_overunder[['Team1', 'Team2']] = kambi_filtered_football_overunder['event_name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
kambi_filtered_football_overunder['OverUnderType2'] = kambi_filtered_football_overunder.apply(
    lambda row: row['participant'] if row['OverUnderType'] in ['Speler schoten op doel', 'Speler schoten'] else (
        '1' if row['Team1'] in row['criterion_english_label'] else 
        '2' if row['Team2'] in row['criterion_english_label'] else 
        'Total team 1 and team 2'
    ),
    axis=1
)

# # Create 'OverUnderType2'
# kambi_filtered_football_overunder['OverUnderType2'] = kambi_filtered_football_overunder.apply(
#     lambda row: '1' if row['Team1'] in row['criterion_english_label'] else (
#         '2' if row['Team2'] in row['criterion_english_label'] else 'Total team 1 and team 2'
#     ),
#     axis=1
# )

# # Create 'line'
# kambi_filtered_football_overunder['line'] = kambi_filtered_football_overunder['line'] / 1000

# # Creat odds
# kambi_filtered_football_overunder['odds'] = kambi_filtered_football_overunder['odds'] / 1000

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/2488754058.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/2488754058.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)
/va

In [85]:
kambi_filtered_football_overunder[(kambi_filtered_football_overunder['OverUnderTime'] != '1e Helft') & (kambi_filtered_football_overunder['OverUnderTime'] != '2e Helft') & (kambi_filtered_football_overunder['OverUnderTime'] != 'Full Time')]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex,OverUnderType,OverUnderTime,Team1,Team2,OverUnderType2


In [86]:
# Create 'OverUnderType'
toto_filtered_football_overunder['OverUnderType'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: 'Goals' if ('Goals' in x and 'Resultaat' not in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
    # else 'Speler schoten op doel' if ('Speler schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
    else 'Wedstrijd schoten op doel' if ('Wedstrijd schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x)
    else 'Wedstrijd schoten' if ('Wedstrijd schoten' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) ## Toto does not seem to offer this. Shame since Bet365 & Kambi have many of these
    else 'Team schoten op doel' if ('Team schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x)
    # else 'Schoten op doel van buiten 16 mtr' if ('schoten op doel van buiten 16 mtr' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) # via specials voor later
    else 'Speler schoten op doel' if ('aantal schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) or ('Speler schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x)
    else 'Speler schoten' if ('aantal schoten' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
    else 'Dubbele Kans' if ('Dubbele Kans' in x or ' en ' in x or ' & ' in x)
    else 'other'
)

# Create 'OverUnderTime'
toto_filtered_football_overunder['OverUnderTime'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: '1e Helft' if '1e helft' in x.lower() else '2e Helft' if '2e helft' in x.lower() 
    else  '00:00 09:59' if 'eerste 10 minuten' in x.lower() else 'Full Time'
)

# Create 'Team1' and 'Team2'
toto_filtered_football_overunder[['Team1', 'Team2']] = toto_filtered_football_overunder['Event Name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder.apply(
    lambda row: (
        '1' if row['OverUnderType'] == 'Goals' and row['Team1'] in row['Market Name'] else
        '2' if row['OverUnderType'] == 'Goals' and row['Team2'] in row['Market Name'] else
        'Total team 1 and team 2' if row['OverUnderType'] == 'Goals' else
        'Total team 1 and team 2' if row['OverUnderType'] == 'Wedstrijd schoten op doel' else
        '1' if row['OverUnderType'] == 'Team schoten op doel' and row['Team1'] in row['Outcome Name'] else
        '2' if row['OverUnderType'] == 'Team schoten op doel' and row['Team2'] in row['Outcome Name'] else
        row['Market Name'].split('aantal schoten')[0].strip() if row['OverUnderType'] in ['Speler schoten op doel', 'Speler schoten'] 
        else 'other'  # Default to OverUnderType if none of the conditions apply
    ),
    axis=1
)


def extract_line_toto(row):
    market_name = row['Market Name']
    outcome_name = row['Outcome Name']
    over_under_type = row['OverUnderType']

    # Extract number from 'Market Name' when it contains 'Over/Under '
    if 'Over/Under ' in market_name:
        return float(market_name.split('Over/Under ')[-1])

    # Extract number from 'Outcome Name'
    match = re.search(r'\d+(\.\d+)?', outcome_name)
    if match:
        value = float(match.group())  # Convert extracted number to float

        # Adjust value based on conditions
        if over_under_type in ['Wedstrijd schoten op doel', 'Team schoten op doel'] and 'of meer' in outcome_name:
            return value - 0.5
        elif over_under_type in ['Speler schoten op doel', 'Speler schoten'] and '+' in outcome_name:
            return value - 0.5
        else:
            return value

    return None  # Default if no number found

# Apply function to create 'line' column
toto_filtered_football_overunder['line'] = toto_filtered_football_overunder.apply(extract_line_toto, axis=1)

# Adjust Outcome Name where {digit} + to over
toto_filtered_football_overunder['Outcome Name'] = toto_filtered_football_overunder['Outcome Name'].apply(
    lambda outcome_name: 'Over' if re.search(r'\d+(\.\d+)?\+', outcome_name) else outcome_name
)

In [87]:
toto_filtered_football_overunder[toto_filtered_football_overunder['OverUnderType'] == 'Wedstrijd schoten']

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex,matched_event,fuzzy_score,OverUnderType,OverUnderTime,Team1,Team2,OverUnderType2,line


In [88]:
kambi_filtered_football_overunder[kambi_filtered_football_overunder['criterion_label'].str.contains('Schoten van Speler ')]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex,OverUnderType,OverUnderTime,Team1,Team2,OverUnderType2
24298,2493092598,1004544421,Schoten van Speler (Afgehandeld Volgens Opta G...,Player's shots (Settled using Opta data),NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022090873,3640331624,Meer dan,Over,1.30,1.5,"Piroe, Joël",OT_OVER,2025-02-17T09:58:08Z,3/10,-335.0,OPEN,ENABLED,NaN,NaN,Leeds United vs Sunderland,FOOTBALL,NaN,2025-02-17T20:00:00Z,M,Speler schoten,Full Time,Leeds United,Sunderland,"Piroe, Joël"
24300,2493092600,1002035662,Schoten van Speler op Doel (Afgehandeld Volgen...,Player's shots on target (Settled using Opta d...,NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022090873,3640331628,Meer dan,Over,1.54,0.5,"Piroe, Joël",OT_OVER,2025-02-17T09:58:08Z,8/15,-186.0,OPEN,ENABLED,NaN,NaN,Leeds United vs Sunderland,FOOTBALL,NaN,2025-02-17T20:00:00Z,M,Speler schoten op doel,Full Time,Leeds United,Sunderland,"Piroe, Joël"
24302,2493092600,1002035662,Schoten van Speler op Doel (Afgehandeld Volgen...,Player's shots on target (Settled using Opta d...,NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022090873,3640331636,Minder dan,Under,2.15,0.5,"Piroe, Joël",OT_UNDER,2025-02-17T09:58:08Z,23/20,115.0,OPEN,ENABLED,NaN,NaN,Leeds United vs Sunderland,FOOTBALL,NaN,2025-02-17T20:00:00Z,M,Speler schoten op doel,Full Time,Leeds United,Sunderland,"Piroe, Joël"
25376,2493412315,1002035662,Schoten van Speler op Doel (Afgehandeld Volgen...,Player's shots on target (Settled using Opta d...,NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022627341,3641477971,Meer dan,Over,2.95,0.5,"Azeez, Femi",OT_OVER,2025-02-15T22:54:29Z,39/20,195.0,OPEN,ENABLED,NaN,NaN,Preston North End vs Millwall,FOOTBALL,NaN,2025-02-18T19:45:00Z,M,Speler schoten op doel,Full Time,Preston North End,Millwall,"Azeez, Femi"
25378,2493412315,1002035662,Schoten van Speler op Doel (Afgehandeld Volgen...,Player's shots on target (Settled using Opta d...,NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022627341,3641478054,Minder dan,Under,1.28,0.5,"Azeez, Femi",OT_UNDER,2025-02-15T22:54:29Z,7/25,-360.0,OPEN,ENABLED,NaN,NaN,Preston North End vs Millwall,FOOTBALL,NaN,2025-02-18T19:45:00Z,M,Speler schoten op doel,Full Time,Preston North End,Millwall,"Azeez, Femi"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190791,2493471495,1002035662,Schoten van Speler op Doel (Afgehandeld Volgen...,Player's shots on target (Settled using Opta d...,NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022723276,3641549080,Minder dan,Under,2.18,0.5,"Marmoush, Omar",OT_UNDER,2025-02-17T07:38:51Z,59/50,118.0,OPEN,ENABLED,NaN,NaN,Real Madrid vs Manchester City,FOOTBALL,NaN,2025-02-19T20:00:00Z,M,Speler schoten op doel,Full Time,Real Madrid,Manchester City,"Marmoush, Omar"
190794,2493471498,1004544421,Schoten van Speler (Afgehandeld Volgens Opta G...,Player's shots (Settled using Opta data),NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022723276,3641549085,Meer dan,Over,1.30,1.5,"Junior, Vinicius",OT_OVER,2025-02-17T07:38:51Z,3/10,-335.0,OPEN,ENABLED,NaN,NaN,Real Madrid vs Manchester City,FOOTBALL,NaN,2025-02-19T20:00:00Z,M,Speler schoten,Full Time,Real Madrid,Manchester City,"Junior, Vinicius"
191339,2493438859,1002035662,Schoten van Speler op Doel (Afgehandeld Volgen...,Player's shots on target (Settled using Opta d...,NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1022727416,3641450264,Meer dan,Over,4.40,0.5,"Pacho, Willian",OT_OVER,2025-02-17T10:18:

In [89]:
# # Function to find the best fuzzy match for OverUnderType2
# def find_best_fuzzy_match(overunder_type2, kambi_df):
#     # Perform fuzzy matching with all Kambi OverUnderType2 values
#     match_scores = kambi_df['OverUnderType2'].apply(lambda x: fuzz.token_set_ratio(overunder_type2, x))
    
#     # Get the best match score and the corresponding OverUnderType2 value
#     best_match_idx = match_scores.idxmax()
#     best_score = match_scores.max()
    
#     if best_score >= 80:  # Threshold for fuzzy matching
#         return kambi_df.loc[best_match_idx, 'OverUnderType2'], best_score
#     return None, None  # No match found

# # Filter records where OverUnderType contains 'Team' or 'Speler'
# filtered_toto = toto_filtered_football_overunder[
#     toto_filtered_football_overunder['OverUnderType'].str.contains('Speler')
# ]
# filtered_kambi = kambi_filtered_football_overunder[
#     kambi_filtered_football_overunder['OverUnderType'].str.contains('Speler')
# ]

# # Apply fuzzy matching function
# filtered_toto[['matched_OverUnderType2', 'fuzzy_score']] = filtered_toto['OverUnderType2'].apply(
#     lambda x: pd.Series(find_best_fuzzy_match(x, filtered_kambi))
# )

# # Merge back to the original dataset (if needed)
# toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
#     filtered_toto[['OverUnderType2', 'matched_OverUnderType2', 'fuzzy_score']],
#     on='OverUnderType2',
#     how='left'
# )

# # Fill OverUnderType2 with matched_OverUnderType2 if it is null then OverUnderType2
# toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder.apply(
#     lambda row: row['OverUnderType2'] if pd.isna(row['matched_OverUnderType2']) else row['matched_OverUnderType2'], axis=1
# )

In [90]:
# # Function to find the best fuzzy match for OverUnderType2
# def find_best_fuzzy_match(overunder_type2, kambi_df):
#     # Perform fuzzy matching with all Kambi OverUnderType2 values
#     match_scores = kambi_df['OverUnderType2'].apply(lambda x: fuzz.token_set_ratio(str(overunder_type2), str(x)))
    
#     # Get the best match score and the corresponding OverUnderType2 value
#     if len(match_scores) > 0:
#         best_match_idx = match_scores.idxmax()
#         best_score = match_scores.max()
        
#         if best_score >= 80:  # Threshold for fuzzy matching
#             return kambi_df.loc[best_match_idx, 'OverUnderType2'], best_score
#     return None, None  # No match found

# # Filter records
# filtered_toto = toto_filtered_football_overunder[
#     toto_filtered_football_overunder['OverUnderType'].str.contains('Speler', na=False)
# ].copy()

# filtered_kambi = kambi_filtered_football_overunder[
#     kambi_filtered_football_overunder['OverUnderType'].str.contains('Speler', na=False)
# ].copy()

# # Apply fuzzy matching function
# fuzzy_matches = filtered_toto['OverUnderType2'].apply(
#     lambda x: find_best_fuzzy_match(x, filtered_kambi)
# )

# filtered_toto['matched_OverUnderType2'] = fuzzy_matches.apply(lambda x: x[0])
# filtered_toto['fuzzy_score'] = fuzzy_matches.apply(lambda x: x[1])

# # Merge back to the original dataset
# toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
#     filtered_toto[['OverUnderType2', 'matched_OverUnderType2', 'fuzzy_score']],
#     on='OverUnderType2',
#     how='left'
# )

# # Fill OverUnderType2 with matched_OverUnderType2 if it is null
# toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder['matched_OverUnderType2'].fillna(toto_filtered_football_overunder['OverUnderType2'])

In [91]:
toto_filtered_football_overunder.columns

Index(['Event Name', 'Market Name', 'Outcome Name', 'Odds (Decimal)',
       'Price Numerator', 'Price Denominator', 'Outcome Type',
       'Outcome SubType', 'sport', 'competition', 'match_name', 'home_team',
       'away_team', 'start_time', 'sex', 'matched_event', 'fuzzy_score',
       'OverUnderType', 'OverUnderTime', 'Team1', 'Team2', 'OverUnderType2',
       'line'],
      dtype='object')

In [92]:
# Function to find the best fuzzy match for OverUnderType2
def find_best_fuzzy_match(row, kambi_df):
    overunder_type2 = row['OverUnderType2']
    start_time = row['start_time']
    
    # First subset based on matching start_time
    time_matched_events = kambi_df[kambi_df['start_time'] == start_time]
    
    if not time_matched_events.empty:
        # Perform fuzzy matching with time-matched Kambi OverUnderType2 values
        match_scores = time_matched_events['OverUnderType2'].apply(
            lambda x: fuzz.token_set_ratio(str(overunder_type2), str(x))
        )
        
        # Get the best match score and the corresponding OverUnderType2 value
        if len(match_scores) > 0:
            best_match_idx = match_scores.idxmax()
            best_score = match_scores.max()
            
            if best_score >= 80:  # Threshold for fuzzy matching
                return time_matched_events.loc[best_match_idx, 'OverUnderType2'], best_score
    return None, None  # No match found

# Filter records
filtered_toto = toto_filtered_football_overunder[
    toto_filtered_football_overunder['OverUnderType'].str.contains('Speler', na=False)
].copy()

filtered_kambi = kambi_filtered_football_overunder[
    kambi_filtered_football_overunder['OverUnderType'].str.contains('Speler', na=False)
].copy()

# Apply fuzzy matching function - now passing both OverUnderType2 and start_time
fuzzy_matches = filtered_toto.apply(
    lambda row: find_best_fuzzy_match(row, filtered_kambi),
    axis=1
)

filtered_toto['matched_OverUnderType2'] = fuzzy_matches.apply(lambda x: x[0])
filtered_toto['fuzzy_score'] = fuzzy_matches.apply(lambda x: x[1])

# Merge back to the original dataset
toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
    filtered_toto[['OverUnderType2', 'matched_OverUnderType2', 'fuzzy_score']],
    on='OverUnderType2',
    how='left'
)

# Fill OverUnderType2 with matched_OverUnderType2 if it is null
toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder['matched_OverUnderType2'].fillna(toto_filtered_football_overunder['OverUnderType2'])

In [93]:
toto_filtered_football_overunder[~toto_filtered_football_overunder['matched_OverUnderType2'].isna()]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex,matched_event,fuzzy_score_x,OverUnderType,OverUnderTime,Team1,Team2,OverUnderType2,line,matched_OverUnderType2,fuzzy_score_y
15874,Barcelona vs Rayo Vallecano,Isi Palazon aantal schoten (Inc Extra Tijd),Over,4.00,3,1,--,NaN,Voetbal,Spanje LaLiga,Barcelona vs Rayo Vallecano,Barcelona,Rayo Vallecano,2025-02-17T20:00:00Z,M,FC Barcelona vs Rayo Vallecano,100.0,Speler schoten,Full Time,Barcelona,Rayo Vallecano,"Palazón, Isi",3.5,"Palazón, Isi",95.0
15875,Barcelona vs Rayo Vallecano,Isi Palazon aantal schoten (Inc Extra Tijd),Over,4.00,3,1,--,NaN,Voetbal,Spanje LaLiga,Barcelona vs Rayo Vallecano,Barcelona,Rayo Vallecano,2025-02-17T20:00:00Z,M,FC Barcelona vs Rayo Vallecano,100.0,Speler schoten,Full Time,Barcelona,Rayo Vallecano,"Palazón, Isi",3.5,"Palazón, Isi",95.0
15876,Barcelona vs Rayo Vallecano,Isi Palazon aantal schoten (Inc Extra Tijd),Over,4.00,3,1,--,NaN,Voetbal,Spanje LaLiga,Barcelona vs Rayo Vallecano,Barcelona,Rayo Vallecano,2025-02-17T20:00:00Z,M,FC Barcelona vs Rayo Vallecano,100.0,Speler schoten,Full Time,Barcelona,Rayo Vallecano,"Palazón, Isi",3.5,"Palazón, Isi",95.0
15877,Barcelona vs Rayo Vallecano,Isi Palazon aantal schoten (Inc Extra Tijd),Over,4.00,3,1,--,NaN,Voetbal,Spanje LaLiga,Barcelona vs Rayo Vallecano,Barcelona,Rayo Vallecano,2025-02-17T20:00:00Z,M,FC Barcelona vs Rayo Vallecano,100.0,Speler schoten,Full Time,Barcelona,Rayo Vallecano,"Palazón, Isi",3.5,"Palazón, Isi",95.0
15878,Barcelona vs Rayo Vallecano,Isi Palazon aantal schoten (Inc Extra Tijd),Over,4.00,3,1,--,NaN,Voetbal,Spanje LaLiga,Barcelona vs Rayo Vallecano,Barcelona,Rayo Vallecano,2025-02-17T20:00:00Z,M,FC Barcelona vs Rayo Vallecano,100.0,Speler schoten,Full Time,Barcelona,Rayo Vallecano,"Palazón, Isi",3.5,"Palazón, Isi",95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52191,Bayern Munchen vs Celtic,Serge Gnabry aantal schoten op doel (Inc Extra...,Over,4.75,15,4,--,NaN,Voetbal,UEFA Champions League,Bayern München vs Celtic,Bayern München,Celtic,2025-02-18T20:00:00Z,M,Bayern Munchen vs Celtic,100.0,Speler schoten op doel,Full Time,Bayern Munchen,Celtic,"Gnabry, Serge",2.5,"Gnabry, Serge",100.0
52192,Bayern Munchen vs Celtic,Serge Gnabry aantal schoten op doel (Inc Extra...,Over,4.75,15,4,--,NaN,Voetbal,UEFA Champions League,Bayern München vs Celtic,Bayern München,Celtic,2025-02-18T20:00:00Z,M,Bayern Munchen vs Celtic,100.0,Speler schoten op doel,Full Time,Bayern Munchen,Celtic,"Gnabry, Serge",2.5,"Gnabry, Serge",100.0
52193,Bayern Munchen vs Celtic,Serge Gnabry aantal schoten op doel (Inc Extra...,Over,4.75,15,4,--,NaN,Voetbal,UEFA Champions League,Bayern München vs Celtic,Bayern München,Celtic,2025-02-18T20:00:00Z,M,Bayern Munchen vs Celtic,100.0,Speler schoten op doel,Full Time,Bayern Munchen,Celtic,"Gnabry, Serge",2.5,"Gnabry, Serge",100.0
52194,Bayern Munchen vs Celtic,Serge Gnabry aantal schoten op doel (Inc Extra...,Over,4.75,15,4,--,NaN,Voetbal,UEFA Champions League,Bayern München vs Celtic,Bayern München,Celtic,2025-02-18T20:00:00Z,M,Bayern Munchen vs Celtic,100.0,Speler schoten op doel,Full Time,Bayern Munchen,Celtic,"Gnabry, Serge",2.5,"Gnabry, Serge",100.0


In [94]:
# # Function to find the best fuzzy match for OverUnderType2
# def find_best_fuzzy_match(overunder_type2, kambi_df):
#     # Perform fuzzy matching with all Kambi OverUnderType2 values
#     match_scores = kambi_df['OverUnderType2'].apply(lambda x: fuzz.token_set_ratio(str(overunder_type2), str(x)))
    
#     # Get the best match score and the corresponding OverUnderType2 value
#     if len(match_scores) > 0:
#         best_match_idx = match_scores.idxmax()
#         best_score = match_scores.max()
        
#         if best_score >= 80:  # Threshold for fuzzy matching
#             return kambi_df.loc[best_match_idx, 'OverUnderType2'], best_score
#     return None, None  # No match found

# # Filter records
# filtered_toto = toto_filtered_football_overunder[
#     toto_filtered_football_overunder['OverUnderType'].str.contains('Speler', na=False)
# ].copy()

# filtered_kambi = kambi_filtered_football_overunder[
#     kambi_filtered_football_overunder['OverUnderType'].str.contains('Speler', na=False)
# ].copy()

# # Apply fuzzy matching function
# fuzzy_matches = filtered_toto['OverUnderType2'].apply(
#     lambda x: find_best_fuzzy_match(x, filtered_kambi)
# )

# filtered_toto['matched_OverUnderType2'] = fuzzy_matches.apply(lambda x: x[0])
# filtered_toto['fuzzy_score'] = fuzzy_matches.apply(lambda x: x[1])


In [95]:
# toto_filtered_football_overunder.to_csv('Data/Testing/toto_filtered_football_overunder.csv')
# kambi_filtered_football_overunder.to_csv('Data/Testing/kambi_filtered_football_overunder.csv')
# merged_football_overunder.to_csv('Data/Testing/merged_football_overunder.csv')


In [96]:
# Perform the merge
merged_football_overunder = pd.merge(
    toto_filtered_football_overunder,
    kambi_filtered_football_overunder,
    left_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'matched_event', 'sex', 'start_time'],
    right_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'event_name', 'sex', 'start_time'],
    how='inner'  # Use 'inner' join to get only matching rows; adjust as needed
)

merged_football_overunder['Outcome Name cleaned'] = merged_football_overunder['Outcome Name'].apply(
    lambda x: 'Over' if re.search(r'\b\d{1,2} of meer\b', x) else x
)

# Keep records with opposite outcomes
merged_football_overunder = merged_football_overunder[merged_football_overunder['outcome_english_label'] != merged_football_overunder['Outcome Name cleaned']] # Dubbel check voor Schoten OverUnderType & voer fuzzymatching uit op speler.

# Display the merged DataFrame
merged_football_overunder.head()

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,match_name,home_team,away_team,start_time,sex,matched_event,fuzzy_score_x,OverUnderType,OverUnderTime,Team1_x,Team2_x,OverUnderType2,line,matched_OverUnderType2,fuzzy_score_y,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport_y,group_name,Team1_y,Team2_y,Outcome Name cleaned
1,Tottenham Hotspur vs Manchester City,Aantal Goals Over/Under 5.5,Over,4.25,13,4,HL,1,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester City,Tottenham Hotspur,Manchester City,2025-02-26T19:30:00Z,M,Tottenham vs Manchester City,100.0,Goals,Full Time,Tottenham Hotspur,Manchester City,Total team 1 and team 2,5.5,NaN,NaN,2494966492,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021042757,3647650922,Minder dan,Under,1.18,NaN,OT_UNDER,2025-02-17T08:33:21Z,1/6,-560.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester City,FOOTBALL,Premier League,Tottenham,Manchester City,Over
2,Tottenham Hotspur vs Manchester City,Aantal Goals Over/Under 5.5,Under,1.18,9,50,HL,L,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester City,Tottenham Hotspur,Manchester City,2025-02-26T19:30:00Z,M,Tottenham vs Manchester City,100.0,Goals,Full Time,Tottenham Hotspur,Manchester City,Total team 1 and team 2,5.5,NaN,NaN,2494966492,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021042757,3647650903,Meer dan,Over,4.80,NaN,OT_OVER,2025-02-17T08:33:21Z,19/5,380.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester City,FOOTBALL,Premier League,Tottenham,Manchester City,Under
4,Tottenham Hotspur vs Manchester City,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,Under,1.12,3,25,HL,L,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester City,Tottenham Hotspur,Manchester City,2025-02-26T19:30:00Z,M,Tottenham vs Manchester City,100.0,Goals,1e Helft,Tottenham Hotspur,Manchester City,1,1.5,NaN,NaN,2494966626,1003194958,Totaal Aantal Doelpunten door Tottenham 1e H...,Total Goals by Tottenham 1st Half,NaN,NaN,6,Over/Onder,Over/Under,1021042757,3647651202,Meer dan,Over,5.10,NaN,OT_OVER,2025-02-17T08:33:19Z,4/1,410.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester City,FOOTBALL,Premier League,Tottenham,Manchester City,Under
7,Tottenham Hotspur vs Manchester City,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,Over,5.50,9,2,HL,1,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester City,Tottenham Hotspur,Manchester City,2025-02-26T19:30:00Z,M,Tottenham vs Manchester City,100.0,Goals,1e Helft,Tottenham Hotspur,Manchester City,1,1.5,NaN,NaN,2494966626,1003194958,Totaal Aantal Doelpunten door Tottenham 1e H...,Total Goals by Tottenham 1st Half,NaN,NaN,6,Over/Onder,Over/Under,1021042757,3647651232,Minder dan,Under,1.13,NaN,OT_UNDER,2025-02-17T08:33:19Z,1/8,-770.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester City,FOOTBALL,Premier League,Tottenham,Manchester City,Over
8,Tottenham Hotspur vs Manchester City,Tottenham Hotspur Aantal Goals Over/Under 1.5,Under,1.65,13,20,HL,L,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester City,Tottenham Hotspur,Manchester City,2025-02-26T19:30:00Z,M,Tottenham vs Manchester City,100.0,Goals,Full Time,Tottenham Hotspur,Manchester City,1,1.5,NaN,NaN,2494966488,1001159967,Totaal Aantal Doelpunten door Tottenham,Total Goals by Tottenham,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021042757,3647650895,Meer dan,Over,2.00,NaN,OT_OVER,2025-02-17T08:33:19Z,Evens,100.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester City,FOOTBALL,Premier League,Tottenham,Manchester City,Under


In [97]:
merged_football_overunder

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,match_name,home_team,away_team,start_time,sex,matched_event,fuzzy_score_x,OverUnderType,OverUnderTime,Team1_x,Team2_x,OverUnderType2,line,matched_OverUnderType2,fuzzy_score_y,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport_y,group_name,Team1_y,Team2_y,Outcome Name cleaned
1,Tottenham Hotspur vs Manchester City,Aantal Goals Over/Under 5.5,Over,4.25,13,4,HL,1,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester City,Tottenham Hotspur,Manchester City,2025-02-26T19:30:00Z,M,Tottenham vs Manchester City,100.0,Goals,Full Time,Tottenham Hotspur,Manchester City,Total team 1 and team 2,5.5,NaN,NaN,2494966492,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021042757,3647650922,Minder dan,Under,1.18,NaN,OT_UNDER,2025-02-17T08:33:21Z,1/6,-560.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester City,FOOTBALL,Premier League,Tottenham,Manchester City,Over
2,Tottenham Hotspur vs Manchester City,Aantal Goals Over/Under 5.5,Under,1.18,9,50,HL,L,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester City,Tottenham Hotspur,Manchester City,2025-02-26T19:30:00Z,M,Tottenham vs Manchester City,100.0,Goals,Full Time,Tottenham Hotspur,Manchester City,Total team 1 and team 2,5.5,NaN,NaN,2494966492,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021042757,3647650903,Meer dan,Over,4.80,NaN,OT_OVER,2025-02-17T08:33:21Z,19/5,380.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester City,FOOTBALL,Premier League,Tottenham,Manchester City,Under
4,Tottenham Hotspur vs Manchester City,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,Under,1.12,3,25,HL,L,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester City,Tottenham Hotspur,Manchester City,2025-02-26T19:30:00Z,M,Tottenham vs Manchester City,100.0,Goals,1e Helft,Tottenham Hotspur,Manchester City,1,1.5,NaN,NaN,2494966626,1003194958,Totaal Aantal Doelpunten door Tottenham 1e H...,Total Goals by Tottenham 1st Half,NaN,NaN,6,Over/Onder,Over/Under,1021042757,3647651202,Meer dan,Over,5.10,NaN,OT_OVER,2025-02-17T08:33:19Z,4/1,410.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester City,FOOTBALL,Premier League,Tottenham,Manchester City,Under
7,Tottenham Hotspur vs Manchester City,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,Over,5.50,9,2,HL,1,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester City,Tottenham Hotspur,Manchester City,2025-02-26T19:30:00Z,M,Tottenham vs Manchester City,100.0,Goals,1e Helft,Tottenham Hotspur,Manchester City,1,1.5,NaN,NaN,2494966626,1003194958,Totaal Aantal Doelpunten door Tottenham 1e H...,Total Goals by Tottenham 1st Half,NaN,NaN,6,Over/Onder,Over/Under,1021042757,3647651232,Minder dan,Under,1.13,NaN,OT_UNDER,2025-02-17T08:33:19Z,1/8,-770.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester City,FOOTBALL,Premier League,Tottenham,Manchester City,Over
8,Tottenham Hotspur vs Manchester City,Tottenham Hotspur Aantal Goals Over/Under 1.5,Under,1.65,13,20,HL,L,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester City,Tottenham Hotspur,Manchester City,2025-02-26T19:30:00Z,M,Tottenham vs Manchester City,100.0,Goals,Full Time,Tottenham Hotspur,Manchester City,1,1.5,NaN,NaN,2494966488,1001159967,Totaal Aantal Doelpunten door Tottenham,Total Goals by Tottenham,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021042757,3647650895,Meer dan,Over,2.00,NaN,OT_OVER,2025-02-17T08:33:19Z,Evens,100.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester City,FOOTBALL,Premier League,Tottenham,Manchester City,Under
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [98]:
kambi_filtered_football_overunder[kambi_filtered_football_overunder['OverUnderType'] == 'Speler schoten op doel']['OverUnderType2'].unique()

array(['Piroe, Joël', 'Azeez, Femi', 'Adebayo, Elijah',
       'Alexander-Arnold, Trent', 'Zanoli, Alessandro',
       'de Oliveira Sertanejo, Daniel "Danielzinho"',
       "dos Santos, 'Guilherme'", 'Oscar', 'Palazón, Isi',
       'Berg, Patrick', 'Ozdoev, Magomed',
       'Aquino Cossa, Eduardo Gabriel "Pepe"', 'Kasius, Denso',
       'Zachariassen, Kristoffer', 'Degreef, Tristan', 'Buksa, Adam',
       'Rasmussen, Christian', 'Hernández, Théo', 'Retegui, Mateo',
       'Pavlidis, Vangelis', 'Gnabry, Serge', 'Gyökeres, Viktor',
       'de Jong, Luuk', 'Marmoush, Omar', 'Pacho, Willian'], dtype=object)

## Asian Handicap

In [99]:
# # filter down to suitable betting opps
# kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

# kambi_filtered_football = kambi_raw_football
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football

# Remove duplicates in toto_filtered_basketbal based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_basketbal based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

In [100]:
kambi_filtered_football_ahandicap = kambi_filtered_football[(kambi_filtered_football['bet_offer_type_name'].str.contains('Asian Handicap'))
]
toto_filtered_football_ahandicap = toto_filtered_football[
    (toto_filtered_football['Market Name'].str.contains('Asian Handicap'))
]

In [101]:
import unicodedata
from fuzzywuzzy import process, fuzz

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_ahandicap['event_name'] = kambi_filtered_football_ahandicap['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_ahandicap['outcome_english_label'] = kambi_filtered_football_ahandicap['outcome_english_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

toto_filtered_football_ahandicap['Event Name'] = toto_filtered_football_ahandicap['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

toto_filtered_football_ahandicap['Outcome Name'] = toto_filtered_football_ahandicap['Outcome Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Create 'Team1' and 'Team2'
kambi_filtered_football_ahandicap[['Team1', 'Team2']] = kambi_filtered_football_ahandicap['event_name'].str.split(' vs ', expand=True)
toto_filtered_football_ahandicap[['Team1', 'Team2']] = toto_filtered_football_ahandicap['Event Name'].str.split(' vs ', expand=True)


# Assign 1 if 'outcome_english_label' matches 'Team1', 2 if it matches 'Team2', else NaN
kambi_filtered_football_ahandicap['outcome_label_simple'] = kambi_filtered_football_ahandicap.apply(
    lambda row: 1 if row['outcome_english_label'] == row['Team1'] 
    else (2 if row['outcome_english_label'] == row['Team2'] else None), axis=1
)

# Assign 1 if 'outcome_english_label' matches 'Team1', 2 if it matches 'Team2', else NaN
toto_filtered_football_ahandicap['outcome_label_simple'] = toto_filtered_football_ahandicap.apply(
    lambda row: 1 if row['Outcome Name'] == row['Team1'] 
    else (2 if row['Outcome Name'] == row['Team2'] else None), axis=1
)

# Extract line from 'Market Name'
toto_filtered_football_ahandicap['line'] = toto_filtered_football_ahandicap['Market Name'].apply(
    lambda x: float(re.findall(r'-?\d*\.?\d+', x)[0]) if re.findall(r'-?\d*\.?\d+', x) else None
)

kambi_events = kambi_filtered_football_ahandicap['event_name'].tolist()

from fuzzywuzzy import fuzz

# Function to find the best match based on Team1 and Team2 fuzzy matching
def find_best_match(event_name, start_time):  # Added start_time parameter
    parts = event_name.split(' vs ')
    if len(parts) == 2:
        team1 = parts[0].strip()
        team2 = parts[1].strip()

        # First subset based on matching start_time
        time_matched_events = kambi_filtered_football_ahandicap[
            kambi_filtered_football_ahandicap['start_time'] == start_time
        ]

        # If we have matching times, then do fuzzy matching on this smaller subset
        if not time_matched_events.empty:
            def match_teams(team, kambi_team):
                return fuzz.token_set_ratio(team, kambi_team)

            # Perform fuzzy matching on the time-matched subset
            kambi_matches = time_matched_events[
                (time_matched_events['Team1'].apply(lambda x: match_teams(team1, x) >= 65)) &
                (time_matched_events['Team2'].apply(lambda x: match_teams(team2, x) >= 65))
            ]

            if not kambi_matches.empty:
                best_score_team1 = kambi_matches['Team1'].apply(lambda x: match_teams(team1, x)).max()
                best_score_team2 = kambi_matches['Team2'].apply(lambda x: match_teams(team2, x)).max()
                average_score = (best_score_team1 + best_score_team2) / 2
                matched_event_name = kambi_matches['event_name'].iloc[0]
                return matched_event_name, average_score

    return None, None

# Modified call to include start_time
toto_filtered_football_ahandicap[['matched_event', 'fuzzy_score']] = toto_filtered_football_ahandicap.apply(
    lambda row: pd.Series(find_best_match(row['Event Name'], row['start_time'])), 
    axis=1
)

# # Apply matching function to filtered_toto_winnaar
# filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

# Check the results
toto_filtered_football_ahandicap[['Event Name', 'matched_event']].head()

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/3109041404.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_ahandicap['event_name'] = kambi_filtered_football_ahandicap['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_86396/3109041404.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_ahandicap['outcome_english_label'] = kambi_filtered_football_ahandicap['outcome_english_label'].apply(
/var/folders/sl/1m_7cj8

,Event Name,matched_event
event_id,,
6617816,FC Utrecht vs NAC,None
6617816,FC Utrecht vs NAC,None
6617816,FC Utrecht vs NAC,None
6617816,FC Utrecht vs NAC,None
6886138,Tottenham Hotspur vs Manchester City,Tottenham vs Manchester City


In [102]:
kambi_filtered_football_ahandicap[kambi_filtered_football_ahandicap['outcome_label_simple'].isna()]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex,Team1,Team2,outcome_label_simple


In [103]:
toto_filtered_football_ahandicap[toto_filtered_football_ahandicap['outcome_label_simple'].isna()]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex,Team1,Team2,outcome_label_simple,line,matched_event,fuzzy_score
event_id,,,,,,,,,,,,,,,,,,,,,


## Match Winnaar & OverUnder and do Arb Analysis

In [104]:
kambi_filtered_football_ahandicap

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex,Team1,Team2,outcome_label_simple
16,2494341954,1002275572,Asian Handicap,Asian Handicap,GOALS,FULL_TIME,7,Asian Handicap,Asian Handicap,1022544815,3645082556,Sporting San Jose FC,Sporting San Jose FC,1.66,-0.25,Sporting San Jose FC,OT_UNTYPED,2025-02-14T22:22:02Z,13/20,-152.0,OPEN,DISABLED,NaN,NaN,Sporting San Jose FC vs Santos de Guapiles,FOOTBALL,Primera Division Costa Rica,2025-02-18T01:00:00Z,M,Sporting San Jose FC,Santos de Guapiles,1
17,2494341954,1002275572,Asian Handicap,Asian Handicap,GOALS,FULL_TIME,7,Asian Handicap,Asian Handicap,1022544815,3645082564,Santos de Guápiles,Santos de Guapiles,2.14,0.25,Santos de Guápiles,OT_UNTYPED,2025-02-14T22:22:02Z,57/50,114.0,OPEN,DISABLED,NaN,NaN,Sporting San Jose FC vs Santos de Guapiles,FOOTBALL,Primera Division Costa Rica,2025-02-18T01:00:00Z,M,Sporting San Jose FC,Santos de Guapiles,2
65,2494341963,1002275573,Asian Handicap – 1e Helft,Asian Handicap - 1st Half,GOALS,NaN,7,Asian Handicap,Asian Handicap,1022544815,3645082607,Sporting San Jose FC,Sporting San Jose FC,2.50,-0.50,Sporting San Jose FC,OT_UNTYPED,2025-02-14T22:22:02Z,6/4,150.0,OPEN,DISABLED,NaN,NaN,Sporting San Jose FC vs Santos de Guapiles,FOOTBALL,Primera Division Costa Rica,2025-02-18T01:00:00Z,M,Sporting San Jose FC,Santos de Guapiles,1
66,2494341963,1002275573,Asian Handicap – 1e Helft,Asian Handicap - 1st Half,GOALS,NaN,7,Asian Handicap,Asian Handicap,1022544815,3645082611,Santos de Guápiles,Santos de Guapiles,1.45,0.50,Santos de Guápiles,OT_UNTYPED,2025-02-14T22:22:02Z,4/9,-225.0,OPEN,DISABLED,NaN,NaN,Sporting San Jose FC vs Santos de Guapiles,FOOTBALL,Primera Division Costa Rica,2025-02-18T01:00:00Z,M,Sporting San Jose FC,Santos de Guapiles,2
179,2494340225,1002275572,Asian Handicap,Asian Handicap,GOALS,FULL_TIME,7,Asian Handicap,Asian Handicap,1022627061,3645074781,Club Guaraní,Club Guarani,1.61,-0.25,Club Guaraní,OT_UNTYPED,2025-02-17T08:51:33Z,3/5,-165.0,OPEN,DISABLED,NaN,NaN,Club Guarani vs Deportivo Recoleta,FOOTBALL,Primera Paraguay,2025-02-21T21:15:00Z,M,Club Guarani,Deportivo Recoleta,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192952,2494473850,1002275573,Asian Handicap – 1e Helft,Asian Handicap - 1st Half,GOALS,NaN,7,Asian Handicap,Asian Handicap,1021350174,3645623544,Sydney FC,Sydney FC,1.60,-0.25,Sydney FC,OT_UNTYPED,2025-02-16T23:12:20Z,3/5,-167.0,OPEN,DISABLED,NaN,NaN,Perth Glory vs Sydney FC,FOOTBALL,A-League,2025-02-22T10:45:00Z,M,Perth Glory,Sydney FC,2
193164,2494894948,1002275572,Asian Handicap,Asian Handicap,GOALS,FULL_TIME,7,Asian Handicap,Asian Handicap,1021350167,3647239681,Western United FC,Western United FC,2.35,-0.50,Western United FC,OT_UNTYPED,2025-02-16T20:47:43Z,27/20,135.0,OPEN,DISABLED,NaN,NaN,Western United FC vs Adelaide United,FOOTBALL,A-League,2025-02-23T06:00:00Z,M,Western United FC,Adelaide United,1
193165,2494894948,1002275572,Asian Handicap,Asian Handicap,GOALS,FULL_TIME,7,Asian Handicap,Asian Handicap,1021350167,3647239685,Adelaide United,Adelaide United,1.55,0.50,Adelaide United,OT_UNTYPED,2025-02-16T20:47:43Z,11/20,-182.0,OPEN,DISABLED,NaN,NaN,Western United FC vs Adelaide United,FOOTBALL,A-League,2025-02-23T06:00:00Z,M,Western United FC,Adelaide United,2
193190,2494894966,1002275573,Asian Handicap – 1e Helft,Asian Handicap - 1st Half,GOALS,NaN,7,Asian Handicap,Asian Handicap,1021350167,3647239725,Western United FC,Western United FC,1.27,0.75,Western United FC,OT_UNTYPED,2025-02-16T22:25:41Z,27/100,-375.0,OPEN,DISABLED,NaN,NaN,Western United FC vs Adelaide United,FOOTBALL,A-League,2025-02-23T06:00:00Z,M,Western Un

In [105]:
# Perform the stacked union
total_football = pd.concat([merged_football_overunder, merged_df_winnaar], ignore_index=True, sort=True)
total_football

,Event Name,Market Name,Odds (Decimal),Outcome Name,Outcome Name cleaned,Outcome SubType,Outcome Type,OverUnderTime,OverUnderType,OverUnderType2,Price Denominator,Price Numerator,Team1_x,Team1_y,Team2_x,Team2_y,away_score,away_team,bet_offer_id,bet_offer_type_english_name,bet_offer_type_id,bet_offer_type_name,cash_out_status,changed_date,competition,criterion_english_label,criterion_id,criterion_label,event_id,event_name,fuzzy_score,fuzzy_score_x,fuzzy_score_y,group_name,home_score,home_team,lifetime,line,match_name,matched_OverUnderType2,matched_event,occurrence_type,odds,odds_american,odds_fractional,outcome_english_label,outcome_id,outcome_label,participant,sex,sport_x,sport_y,standardized_label,start_time,status,type
0,Tottenham Hotspur vs Manchester City,Aantal Goals Over/Under 5.5,4.25,Over,Over,1,HL,Full Time,Goals,Total team 1 and team 2,4,13,Tottenham Hotspur,Tottenham,Manchester City,Manchester City,NaN,Manchester City,2.494966e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-17T08:33:21Z,Engeland Premier League,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021043e+09,Tottenham vs Manchester City,NaN,100.0,NaN,Premier League,NaN,Tottenham Hotspur,FULL_TIME,5.5,Tottenham Hotspur vs Manchester City,NaN,Tottenham vs Manchester City,GOALS,1.18,-560.0,1/6,Under,3.647651e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-26T19:30:00Z,OPEN,OT_UNDER
1,Tottenham Hotspur vs Manchester City,Aantal Goals Over/Under 5.5,1.18,Under,Under,L,HL,Full Time,Goals,Total team 1 and team 2,50,9,Tottenham Hotspur,Tottenham,Manchester City,Manchester City,NaN,Manchester City,2.494966e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-17T08:33:21Z,Engeland Premier League,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021043e+09,Tottenham vs Manchester City,NaN,100.0,NaN,Premier League,NaN,Tottenham Hotspur,FULL_TIME,5.5,Tottenham Hotspur vs Manchester City,NaN,Tottenham vs Manchester City,GOALS,4.80,380.0,19/5,Over,3.647651e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-26T19:30:00Z,OPEN,OT_OVER
2,Tottenham Hotspur vs Manchester City,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,1.12,Under,Under,L,HL,1e Helft,Goals,1,25,3,Tottenham Hotspur,Tottenham,Manchester City,Manchester City,NaN,Manchester City,2.494967e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-17T08:33:19Z,Engeland Premier League,Total Goals by Tottenham 1st Half,1.003195e+09,Totaal Aantal Doelpunten door Tottenham 1e H...,1.021043e+09,Tottenham vs Manchester City,NaN,100.0,NaN,Premier League,NaN,Tottenham Hotspur,NaN,1.5,Tottenham Hotspur vs Manchester City,NaN,Tottenham vs Manchester City,NaN,5.10,410.0,4/1,Over,3.647651e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-26T19:30:00Z,OPEN,OT_OVER
3,Tottenham Hotspur vs Manchester City,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,5.50,Over,Over,1,HL,1e Helft,Goals,1,2,9,Tottenham Hotspur,Tottenham,Manchester City,Manchester City,NaN,Manchester City,2.494967e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-17T08:33:19Z,Engeland Premier League,Total Goals by Tottenham 1st Half,1.003195e+09,Totaal Aantal Doelpunten door Tottenham 1e H...,1.021043e+09,Tottenham vs Manchester City,NaN,100.0,NaN,Premier League,NaN,Tottenham Hotspur,NaN,1.5,Tottenham Hotspur vs Manchester City,NaN,Tottenham vs Manchester City,NaN,1.13,-770.0,1/8,Under,3.647651e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-26T19:30:00Z,OPEN,OT_UNDER
4,Tottenham Hotspur vs Manchester City,Tottenham Hotspur Aantal Goals Over/Under 1.5,1.65,Under,Under,L,HL,Full Time,Goals,1,20,13,Tottenham Hotspur,Tottenham,Manchester City,Manchester City,NaN,Manchester City,2.494966e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-17T08:33:19Z,Engeland Premier League,Total Goals by Tottenham,1.001160e+09,Totaal Aantal Doelpunten door Tottenham,1.021043e+09,Tottenham vs Manchester City,NaN,100.0,NaN,Premier League,NaN,Tottenham Hotspur,FULL_TIME,1.5,Tottenham Hotspur vs Manchester City,NaN,Tottenham vs Manchester City,NaN,2.00,100.0,Evens,Over,3.647651e+09,Meer dan,NaN,M,Voetbal,FOOTBAL

In [106]:
# Calculate Arbitrage Percentage
total_football['Arbitrage Percentage'] = (1 / total_football['Odds (Decimal)'] + 1 / total_football['odds']) * 100

# Identify Arbitrage Opportunities
total_football['Is Arbitrage'] = total_football['Arbitrage Percentage'] < 100

# Calculate Optimal Stakes if Arbitrage
total_stake = 1000
total_football['Stake A'] = total_football.apply(
    lambda row: (1 / row['Odds (Decimal)'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)
total_football['Stake B'] = total_football.apply(
    lambda row: (1 / row['odds'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)

total_football[['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]    

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
0,Tottenham Hotspur vs Manchester City,Aantal Goals Over/Under 5.5,Over,Minder dan,4.25,1.18,108.275174,False,0,0
1,Tottenham Hotspur vs Manchester City,Aantal Goals Over/Under 5.5,Under,Meer dan,1.18,4.80,105.579096,False,0,0
2,Tottenham Hotspur vs Manchester City,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,Under,Meer dan,1.12,5.10,108.893557,False,0,0
3,Tottenham Hotspur vs Manchester City,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,Over,Minder dan,5.50,1.13,106.677393,False,0,0
4,Tottenham Hotspur vs Manchester City,Tottenham Hotspur Aantal Goals Over/Under 1.5,Under,Meer dan,1.65,2.00,110.606061,False,0,0
...,...,...,...,...,...,...,...,...,...,...
6308,Lamontville Golden Arrows vs Orlando Pirates,1e Helft: Draw No Bet,Lamontville Golden Arrows,2,3.50,1.24,109.216590,False,0,0
6309,Polonia Warszawa vs Gornik Leczna,1e Helft: Draw No Bet,Polonia Warszawa,2,1.59,2.08,110.970005,False,0,0
6310,Polonia Warszawa vs Gornik Leczna,1e Helft: Draw No Bet,Gornik Leczna,1,2.10,1.65,108.225108,False,0,0
6311,Polonia Warszawa vs Gornik Leczna,2e Helft: Draw No Bet,Polonia Warszawa,2,1.61,2.08,110.188724,False,0,0


In [107]:
# Count unique combinations of OverUnderType1 and OverUnderType2
combination_counts = total_football.groupby(['OverUnderType', 'OverUnderType2']).size().reset_index(name='count')

# Display the result
print(combination_counts)

                OverUnderType           OverUnderType2  count
0                       Goals                        1   1058
1                       Goals                        2   1156
2                       Goals  Total team 1 and team 2   1776
3      Speler schoten op doel            Gnabry, Serge     10
4      Speler schoten op doel          Hernández, Théo     10
5      Speler schoten op doel             Palazón, Isi     10
6      Speler schoten op doel       Pavlidis, Vangelis     11
7      Speler schoten op doel           Retegui, Mateo     11
8      Speler schoten op doel       Zanoli, Alessandro      7
9        Team schoten op doel                        1      1
10       Team schoten op doel                        2      1
11  Wedstrijd schoten op doel  Total team 1 and team 2      2


In [108]:
total_football_py = pd.read_csv('test_total_merge_Football_2025-02-10 11:14:13.074775.csv')
# Count unique combinations of OverUnderType1 and OverUnderType2
combination_counts = total_football_py.groupby(['OverUnderType', 'OverUnderType2']).size().reset_index(name='count')

# Display the result
print(combination_counts)

                 OverUnderType           OverUnderType2  count
0                        Goals                        1   2954
1                        Goals                        2   2884
2                        Goals  Total team 1 and team 2   6725
3       Speler schoten op doel           Abraham, Tammy      1
4       Speler schoten op doel           Adeyemi, Karim      1
..                         ...                      ...    ...
191     Speler schoten op doel          de Roon, Marten      1
192     Speler schoten op doel          de Vrij, Stefan      1
193       Team schoten op doel                        1      4
194       Team schoten op doel                        2      2
195  Wedstrijd schoten op doel  Total team 1 and team 2      4

[196 rows x 3 columns]


In [120]:
total_football.sort_values(by='Arbitrage Percentage', ascending=True)

,Event Name,Market Name,Odds (Decimal),Outcome Name,Outcome Name cleaned,Outcome SubType,Outcome Type,OverUnderTime,OverUnderType,OverUnderType2,Price Denominator,Price Numerator,Team1_x,Team1_y,Team2_x,Team2_y,away_score,away_team,bet_offer_id,bet_offer_type_english_name,bet_offer_type_id,bet_offer_type_name,cash_out_status,changed_date,competition,criterion_english_label,criterion_id,criterion_label,event_id,event_name,fuzzy_score,fuzzy_score_x,fuzzy_score_y,group_name,home_score,home_team,lifetime,line,match_name,matched_OverUnderType2,matched_event,occurrence_type,odds,odds_american,odds_fractional,outcome_english_label,outcome_id,outcome_label,participant,sex,sport_x,sport_y,standardized_label,start_time,status,type,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
3132,Paris Saint Germain FC vs Stade Brest,Aantal Goals Over/Under 0.5,1.01,Over,Over,1,HL,Full Time,Goals,Total team 1 and team 2,100,1,Paris Saint Germain FC,Paris SG,Stade Brest,Brest,NaN,Stade Brest,2.493439e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-17T10:18:43Z,UEFA Champions League,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.022727e+09,Paris SG vs Brest,NaN,88.5,NaN,NaN,NaN,Paris Saint Germain FC,FULL_TIME,0.5,Paris Saint Germain FC vs Stade Brest,NaN,Paris SG vs Brest,GOALS,30.00,2900.0,29/1,Under,3.641450e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-19T20:00:00Z,OPEN,OT_UNDER,102.343234,False,0,0
2886,Benfica vs AS Monaco,Aantal Goals Over/Under 0.5,1.03,Over,Over,1,HL,Full Time,Goals,Total team 1 and team 2,100,3,Benfica,Benfica,AS Monaco,AS Monaco,NaN,AS Monaco,2.493785e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-17T10:11:56Z,UEFA Champions League,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.022759e+09,Benfica vs AS Monaco,NaN,100.0,NaN,NaN,NaN,Benfica,FULL_TIME,0.5,Benfica vs AS Monaco,NaN,Benfica vs AS Monaco,GOALS,19.00,1800.0,18/1,Under,3.642811e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-18T20:00:00Z,OPEN,OT_UNDER,102.350537,False,0,0
2902,Jagiellonia Bialystok vs FK Tsc Backa Topola,Aantal Goals Over/Under 4.5,1.23,Under,Under,L,HL,Full Time,Goals,Total team 1 and team 2,100,23,Jagiellonia Bialystok,Jagiellonia Białystok,FK Tsc Backa Topola,TSC Backa Topola,NaN,FK Tsc Backa Topola,2.494208e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-17T07:41:01Z,UEFA Europa Conference League,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.022384e+09,Jagiellonia Białystok vs TSC Backa Topola,NaN,97.5,NaN,NaN,NaN,Jagiellonia Bialystok,FULL_TIME,4.5,Jagiellonia Bialystok vs FK Tsc Backa Topola,NaN,Jagiellonia Białystok vs TSC Backa Topola,GOALS,4.75,375.0,15/4,Over,3.644503e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-20T20:00:00Z,OPEN,OT_OVER,102.353445,False,0,0
4043,Lamontville Golden Arrows vs Orlando Pirates,Aantal Goals Over/Under 5.5,1.01,Under,Under,L,HL,Full Time,Goals,Total team 1 and team 2,100,1,Lamontville Golden Arrows,Golden Arrows,Orlando Pirates,Orlando Pirates,NaN,Orlando Pirates,2.493658e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-14T06:51:20Z,Zuid-Afrika Premier Soccer League,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.021389e+09,Golden Arrows vs Orlando Pirates,NaN,100.0,NaN,SA Premier Soccer League,NaN,Lamontville Golden Arrows,FULL_TIME,5.5,Lamontville Golden Arrows vs Orlando Pirates,NaN,Golden Arrows vs Orlando Pirates,GOALS,29.00,2800.0,28/1,Over,3.642475e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-19T17:30:00Z,OPEN,OT_OVER,102.458177,False,0,0
2726,Atalanta vs Club Brugge,Aantal Goals Over/Under 4.5,1.20,Under,Under,L,HL,Full Time,Goals,Total team 1 and team 2,5,1,Atalanta,Atalanta,Club Brugge,Club Brugge,NaN,Club Brugge,2.493772e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-17T10:06:00Z,UEFA Champions League,Total Goals,1.001160e+09,Totaal Aantal Doelpunten,1.022759e+09,Atalanta vs Club Brugge,NaN,100.0,NaN,NaN,NaN,Atalanta,FULL_TIME,4.5,Atalanta vs Club Brugge,NaN,Atalanta vs Club Brugge,GOALS,5.20,420.0,21/5,Over,3.642849e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-18T20:00:00Z

In [112]:
total_football[total_football['Is Arbitrage'] == True][['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]  

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B


In [113]:
total_football[(total_football['Is Arbitrage'] == True)][['Event Name', 'matched_event', 'Market Name', 'criterion_label', 'Outcome Name', 'Outcome Name cleaned', 'line', 'outcome_english_label', 'type', 'Outcome SubType', 'OverUnderType2',
       'Odds (Decimal)', 'odds', 'Price Numerator', 'Price Denominator',
       'Outcome Type', 'Outcome SubType', 'sport_x', 'competition',
       'OverUnderType', 'OverUnderType2', 'OverUnderTime', 'Team1_x', 'Team2_x', 'Team1_y', 'Team2_y', 'Arbitrage Percentage',
       'Is Arbitrage', 'Stake A', 'Stake B', 'group_name', 'sex', 'start_time']]

,Event Name,matched_event,Market Name,criterion_label,Outcome Name,Outcome Name cleaned,line,outcome_english_label,type,Outcome SubType,OverUnderType2,Odds (Decimal),odds,Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,OverUnderType,OverUnderType2,OverUnderTime,Team1_x,Team2_x,Team1_y,Team2_y,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B,group_name,sex,start_time


In [114]:
total_football[total_football['OverUnderType'] == 'Speler aantal schoten op doel'][['Event Name', 'matched_event', 'Market Name', 'criterion_label', 'Outcome Name', 'Outcome Name cleaned', 'line', 'outcome_english_label', 'type', 'Outcome SubType', 'OverUnderType2',
       'Odds (Decimal)', 'odds', 'Price Numerator', 'Price Denominator',
       'Outcome Type', 'Outcome SubType', 'sport_x', 'competition',
       'OverUnderType', 'OverUnderTime', 'Team1_x', 'Team2_x', 'Team1_y', 'Team2_y', 'Arbitrage Percentage',
       'Is Arbitrage', 'Stake A', 'Stake B', 'group_name', 'sex', 'start_time']]

,Event Name,matched_event,Market Name,criterion_label,Outcome Name,Outcome Name cleaned,line,outcome_english_label,type,Outcome SubType,OverUnderType2,Odds (Decimal),odds,Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,OverUnderType,OverUnderTime,Team1_x,Team2_x,Team1_y,Team2_y,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B,group_name,sex,start_time


In [115]:
# # Function to find the best fuzzy match for OverUnderType2
# def find_best_fuzzy_match(overunder_type2, kambi_df):
#     # Perform fuzzy matching with all Kambi OverUnderType2 values
#     match_scores = kambi_df['OverUnderType2'].apply(lambda x: fuzz.token_set_ratio(overunder_type2, x))
    
#     # Get the best match score and the corresponding OverUnderType2 value
#     best_match_idx = match_scores.idxmax()
#     best_score = match_scores.max()
    
#     if best_score >= 65:  # Threshold for fuzzy matching
#         return kambi_df.loc[best_match_idx, 'OverUnderType2'], best_score
#     return None, None  # No match found

# # Filter records where OverUnderType contains 'Team' or 'Speler'
# filtered_toto = toto_filtered_football_overunder[
#     toto_filtered_football_overunder['OverUnderType'].str.contains('Speler')
# ]
# filtered_kambi = kambi_filtered_football_overunder[
#     kambi_filtered_football_overunder['OverUnderType'].str.contains('Speler')
# ]

# # Apply fuzzy matching function
# filtered_toto[['matched_OverUnderType2', 'fuzzy_score']] = filtered_toto['OverUnderType2'].apply(
#     lambda x: pd.Series(find_best_fuzzy_match(x, filtered_kambi))
# )

# # Merge back to the original dataset (if needed)
# toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
#     filtered_toto[['OverUnderType2', 'matched_OverUnderType2', 'fuzzy_score']],
#     on='OverUnderType2',
#     how='left'
# )

In [116]:
# Fill matched_OverUnderType2 with OverUnderType if it is null
toto_filtered_football_overunder['final_OverUnderType'] = toto_filtered_football_overunder.apply(
    lambda row: row['OverUnderType'] if pd.isna(row['matched_OverUnderType2']) else row['matched_OverUnderType2'], axis=1
)

In [117]:
toto_filtered_football_overunder[toto_filtered_football_overunder['matched_OverUnderType2'].isna()]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex,matched_event,fuzzy_score_x,OverUnderType,OverUnderTime,Team1,Team2,OverUnderType2,line,matched_OverUnderType2,fuzzy_score_y,final_OverUnderType
0,FC Utrecht vs NAC,Aantal Goals Over/Under 2.5,Over,1.49,49,100,HL,1,Voetbal,Nederland Eredivisie,FC Utrecht vs NAC,FC Utrecht,NAC,2025-03-01T15:30:00Z,M,None,NaN,Goals,Full Time,FC Utrecht,NAC,Total team 1 and team 2,2.5,NaN,NaN,Goals
1,FC Utrecht vs NAC,Aantal Goals Over/Under 2.5,Under,2.45,29,20,HL,L,Voetbal,Nederland Eredivisie,FC Utrecht vs NAC,FC Utrecht,NAC,2025-03-01T15:30:00Z,M,None,NaN,Goals,Full Time,FC Utrecht,NAC,Total team 1 and team 2,2.5,NaN,NaN,Goals
2,FC Utrecht vs NAC,FC Utrecht Aantal Goals Over/Under 1.5,Over,1.44,11,25,HL,1,Voetbal,Nederland Eredivisie,FC Utrecht vs NAC,FC Utrecht,NAC,2025-03-01T15:30:00Z,M,None,NaN,Goals,Full Time,FC Utrecht,NAC,1,1.5,NaN,NaN,Goals
3,FC Utrecht vs NAC,FC Utrecht Aantal Goals Over/Under 1.5,Under,2.50,6,4,HL,L,Voetbal,Nederland Eredivisie,FC Utrecht vs NAC,FC Utrecht,NAC,2025-03-01T15:30:00Z,M,None,NaN,Goals,Full Time,FC Utrecht,NAC,1,1.5,NaN,NaN,Goals
4,FC Utrecht vs NAC,NAC Aantal Goals Over/Under 1.5,Over,3.15,43,20,HL,1,Voetbal,Nederland Eredivisie,FC Utrecht vs NAC,FC Utrecht,NAC,2025-03-01T15:30:00Z,M,None,NaN,Goals,Full Time,FC Utrecht,NAC,2,1.5,NaN,NaN,Goals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75110,Norrkoping vs Osters,Norrkoping 1e Helft: Aantal Goals Over/Under...,Over,1.71,71,100,HL,1,Voetbal,Zweden Allsvenskan,Norrköping vs Östers,Norrköping,Östers,2025-03-30T14:30:00Z,M,None,NaN,Goals,1e Helft,Norrkoping,Osters,1,0.5,NaN,NaN,Goals
75111,Norrkoping vs Osters,Osters 2e Helft: Aantal Goals Over/Under 0.5,Under,1.77,77,100,HL,L,Voetbal,Zweden Allsvenskan,Norrköping vs Östers,Norrköping,Östers,2025-03-30T14:30:00Z,M,None,NaN,Goals,2e Helft,Norrkoping,Osters,2,0.5,NaN,NaN,Goals
75112,Norrkoping vs Osters,Osters 2e Helft: Aantal Goals Over/Under 0.5,Over,1.95,19,20,HL,1,Voetbal,Zweden Allsvenskan,Norrköping vs Östers,Norrköping,Östers,2025-03-30T14:30:00Z,M,None,NaN,Goals,2e Helft,Norrkoping,Osters,2,0.5,NaN,NaN,Goals
75113,Norrkoping vs Osters,1e Helft: Aantal Goals Over/Under 2.5,Over,5.25,17,4,HL,1,Voetbal,Zweden Allsvenskan,Norrköping vs Östers,Norrköping,Östers,2025-03-30T14:30:00Z,M,None,NaN,Goals,1e Helft,Norrkoping,Osters,Total team 1 and team 2,2.5,NaN,NaN,Goals
